# Benchmarking
benchmarking using [MQTBench](https://mqt.readthedocs.io/projects/bench/en/latest/Quickstart.html) with QFT for 15 qubits

In [ ]:
from lib.interaction_mapping import InteractionMapping

In [1]:
from mqt.bench import get_benchmark

In [ ]:
from qiskit import QuantumCircuit